In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import CCA
from CCA.utils import *
import os
import scipy

In [2]:
Ypc = np.load('/data/NIMH_scratch/zwallymi/behavioral/new_regressed_behavioral_pca.npy')

In [3]:
dir = '/data/NIMH_scratch/zwallymi/gradients/individual_files/parcellated/CCA_site_regressed_gradients'
files = os.listdir(dir)
gradients = [np.load(f'{dir}/{f}') for f in files]
subjects = [x[4:8]+'_'+x[8:19] for x in files]

In [4]:
flat_gradients = []
for grad in gradients :
    flat = np.hstack(grad)
    flat_gradients.append(flat)
all_gradients = np.array(flat_gradients)

In [5]:
PCs, vari_explain, s = pca_wrap(all_gradients, method='vanilla', reg=1, ridge_reg=0.1)

In [ ]:
vari_explain[:20]

In [7]:
def seber_cca(Y, X, R, S):
    N = Y.shape[0]
    Qy, Ry, iY = scipy.linalg.qr((Y), mode='economic', pivoting=True)
    Qx, Rx, iX = scipy.linalg.qr((X), mode='economic', pivoting=True)
    K = min(np.linalg.matrix_rank(Y), np.linalg.matrix_rank(X))
    QyTQx = Qy.T @ Qx
    if K <= 6 or K == np.min(QyTQx.shape):
        L, D, MT = scipy.linalg.svd(QyTQx)
    else:
        L, D, MT = scipy.sparse.linalg.svds(QyTQx, k=K)
    
    cc = np.minimum(np.maximum(D[:K], 0), 1)
    A = np.linalg.pinv(Ry) @ (L[:, :K]) * np.sqrt(N - R)
    B = np.linalg.pinv(Rx) @ (MT[:K, :].T) * np.sqrt(N - S)
    A = A[iY]
    B = B[iX]
    return A, B, cc

def center(X):
    icte = np.sum(np.diff(X, axis=0) ** 2, axis=0) == 0
    X = np.delete(X, np.where(icte), axis=1)
    X = X - np.mean(X, axis=0)
    return X

In [ ]:
A, B, cc = seber_cca(center(Ypc), center(all_gradients), 2, 2)
print(cc)

In [ ]:
from CCA.permcca import permcca
p, r, A, B, U, V, perm_dist = permcca(Ypc, all_gradients, 5)